In [ ]:
from similarity_search_gui import load_images_features, find_similar_images
from misc import load, load_class_labels
from data_loader import data_loader;
import numpy as np
from tqdm import tqdm

dataset_root = "../dataset/"
test_info = load(dataset_root + 'labels/test.csv')
test_dir = dataset_root + 'val_set/'

dl_test = data_loader(test_info, test_dir, 1, (224, 224));

In [ ]:
nn, _ = load_images_features(False)

def evaluate(base_weight):
    results = []
    min_cut = 60;

    for i in tqdm(range(dl_test.number_of_batch())):
        _, labels, filenames = dl_test.get_batch(i);
        query_label = labels[0];
        query_filename = filenames[0];
        query_filename = dataset_root + 'complete/' + query_filename;

        most_similar_filenames = find_similar_images(query_filename, None, nn, False, True, base_weight=base_weight, verbose=0);
        most_similar_filenames = np.array([file.split('/')[-1] for file in most_similar_filenames]);
        mask = [('val' in file) for file in most_similar_filenames];
        most_similar_filenames = most_similar_filenames[mask];

        cut_point = min(len(test_info[test_info['label'] == query_label].values), min_cut)
        most_similar_filenames = most_similar_filenames[:cut_point];
        
        correct = 0
        wrong = 0 
        for file in most_similar_filenames:
            label_found = test_info[test_info['filename'] == file]['label'].values[0]
            if label_found == query_label:
                correct += 1;
            else:
                wrong += 1
        results.append((correct/(correct + wrong), (correct + wrong)))
        #if i % 10 == 0:
        #    print(correct, wrong)

        total_element = 0
        acc = 0
        for res in results:
            perc, weight = res;
            total_element += weight;
            acc += perc * weight;
        acc /= total_element
            
    return results, acc
        

## Evaluation with only efficient net base

In [ ]:
res, acc = evaluate(1)
print(acc)

## Evaluation with ensembling model

In [ ]:
res, acc = evaluate(0)
print(acc)